In [14]:
    if __name__ == '__main__':
        %load_ext loader
    else:
        %load_ext rites.loader
    from rites.config import alias
        
    [alias.__setitem__(str, f"rites.{str}") for str in ('markdown', 'template', 'conventions', 'test')]

The loader extension is already loaded. To reload it, use:
  %reload_ext loader


[None, None, None, None]

In [15]:
    def load_ipython_extension(ip):
        if __name__ == '__main__':
            %reload_ext config
        else:
            %load_ext rites.config

In [ ]:
    if __name__ == '__main__':
        !jupyter nbconvert --to python __init__.ipynb
        load_ipython_extension(__import__('IPython').get_ipython())